In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [3]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew

load_dotenv()  # charge .env
os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME", "gpt-3.5-turbo")

# test simple
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're planning a blog article about {topic}.",
    allow_delegation=False,
    verbose=True
)
print("OK: Agent prêt ✅")

OK: Agent prêt ✅


In [7]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [11]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [12]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [13]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [15]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the current landscape of Artificial Intelligence (AI) technology
- Mention of the rapid advancements and widespread adoption of AI in various industries

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning algorithms
- Natural language processing
- Robotics and automation
- AI ethics and bias
- AI in healthcare, finance, and marketing

2. Key Players in th

In [19]:
from IPython.display import Markdown
from openai import OpenAI

client = OpenAI()

def translate(text):
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role":"system", "content":"Tu es un traducteur professionnel. Traduis en français sans perdre le style."},
            {"role":"user", "content": text}
        ]
    ).choices[0].message.content

# 1. Planner
outline = plan.execute(agent=planner, context={"topic": "Artificial Intelligence"})
print("=== PLAN (EN) ===")
display(Markdown(outline))
print("\n=== PLAN (FR) ===")
display(Markdown(translate(outline)))

# 2. Writer
draft = write.execute(agent=writer, context={"topic": "Artificial Intelligence"})
print("\n=== DRAFT (EN) ===")
display(Markdown(draft))
print("\n=== DRAFT (FR) ===")
display(Markdown(translate(draft)))

# 3. Editor
final = edit.execute(agent=editor, context={"topic": "Artificial Intelligence"})
print("\n=== FINAL (EN) ===")
display(Markdown(final))
print("\n=== FINAL (FR) ===")
display(Markdown(translate(final)))



> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: Artificial Intelligence

Topic: Artificial Intelligence

Audience Analysis:
Target Audience: Professionals in the tech industry, business owners looking to implement AI, students studying AI, and anyone interested in the latest trends in technology.
Interests: Cutting-edge technology, innovation, machine learning, automation, data analysis.
Pain Points: Understanding complex AI concepts, staying updated on the latest trends, implementing AI in business effectively.

SEO Keywords:
1. Artificial Intelligence trends
2. Machine learning advancements
3. AI in business
4. Data analysis with AI
5. Future of artificial intelligence

Content Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in various industries
- Overview of the latest trends in AI

II. Key Points
A. Latest Trends in AI
1. Advancements in machine learning algorithms
2. Integration of AI in he

Content Plan: Artificial Intelligence

Topic: Artificial Intelligence

Audience Analysis:
Target Audience: Professionals in the tech industry, business owners looking to implement AI, students studying AI, and anyone interested in the latest trends in technology.
Interests: Cutting-edge technology, innovation, machine learning, automation, data analysis.
Pain Points: Understanding complex AI concepts, staying updated on the latest trends, implementing AI in business effectively.

SEO Keywords:
1. Artificial Intelligence trends
2. Machine learning advancements
3. AI in business
4. Data analysis with AI
5. Future of artificial intelligence

Content Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in various industries
- Overview of the latest trends in AI

II. Key Points
A. Latest Trends in AI
1. Advancements in machine learning algorithms
2. Integration of AI in healthcare and finance
3. Ethical considerations in AI development
4. Impact of AI on job markets

B. Key Players in AI
1. Leading companies in AI technology
2. Notable AI researchers and experts
3. AI startups making a significant impact

C. Noteworthy News in AI
1. Recent breakthroughs in AI research
2. AI applications in solving real-world problems
3. Events and conferences related to AI

III. Call to Action
- Encourage readers to stay informed about AI developments
- Provide resources for further learning and exploration in AI
- Promote engagement through social media channels

Resources:
1. Forbes - Artificial Intelligence News
2. MIT Technology Review - AI section
3. Towards Data Science - AI articles
4. World Economic Forum - AI and Automation reports
5. Google AI Blog - Updates on Google's AI projects

By following this content plan, the blog article on Artificial Intelligence will provide valuable insights to the target audience, helping them stay informed and make informed decisions regarding AI technologies.


=== PLAN (FR) ===


Plan de contenu : Intelligence Artificielle

Sujet : Intelligence Artificielle

Analyse de l'audience :
Public cible : Professionnels de l'industrie technologique, chefs d'entreprise intéressés par la mise en œuvre de l'IA, étudiants en IA, et toute personne intéressée par les dernières tendances technologiques.
Intérêts : Technologie de pointe, innovation, apprentissage automatique, automatisation, analyse de données.
Points de douleur : Compréhension des concepts complexes de l'IA, rester informé des dernières tendances, mise en place efficace de l'IA dans les entreprises.

Mots-clés SEO :
1. Tendances en Intelligence Artificielle
2. Avancées en apprentissage automatique
3. IA dans les entreprises
4. Analyse de données avec IA
5. Avenir de l'intelligence artificielle

Plan de contenu :
I. Introduction
- Définition de l'Intelligence Artificielle
- Importance de l'IA dans diverses industries
- Aperçu des dernières tendances en IA

II. Points clés
A. Dernières tendances en IA
1. Progrès dans les algorithmes d'apprentissage machine
2. Intégration de l'IA dans le domaine de la santé et de la finance
3. Considérations éthiques dans le développement de l'IA
4. Impact de l'IA sur les marchés du travail

B. Acteurs clés en IA
1. Entreprises leaders en technologie IA
2. Chercheurs et experts notables en IA
3. Startups en IA ayant un impact significatif

C. Actualités importantes en IA
1. Dernières percées dans la recherche en IA
2. Applications de l'IA pour résoudre des problèmes concrets
3. Événements et conférences liés à l'IA

III. Appel à l'action
- Encourager les lecteurs à rester informés des développements en IA
- Fournir des ressources pour approfondir la compréhension de l'IA et explorer davantage
- Promouvoir l'engagement via les réseaux sociaux

Ressources :
1. Forbes - Actualités sur l'Intelligence Artificielle
2. MIT Technology Review - Section sur l'IA
3. Towards Data Science - Articles sur l'IA
4. Forum économique mondial - Rapports sur l'IA et l'automatisation
5. Google AI Blog - Mises à jour sur les projets d'IA de Google

En suivant ce plan de contenu, l'article de blog sur l'Intelligence Artificielle fournira des perspectives précieuses au public cible, les aidant à rester informés et à prendre des décisions éclairées concernant les technologies de l'IA.



> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
# Unleashing the Power of Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has been a revolutionary force in the world of technology, transforming industries and reshaping the way we live and work. With the ability to analyze vast amounts of data, learn from patterns, and make autonomous decisions, AI has the potential to unlock new levels of efficiency, productivity, and innovation. In this blog post, we will delve into the fascinating world of Artificial Intelligence, exploring its applications, impact, and future prospects.

## The Impact of AI
AI has already made significant strides in various fields, from healthcare and finance to transportation and entertainment. In healthcare, AI-powered systems can analyze medical images, predict patient outcomes, and assist in diagnosis and treatment planning. In finance, AI algorithms can analyze market trends, optimize investment por

...

# Libérer la puissance de l'Intelligence Artificielle

## Introduction
L'Intelligence Artificielle (IA) a été une force révolutionnaire dans le monde de la technologie, transformant les industries et remodelant notre manière de vivre et de travailler. Avec la capacité d'analyser de vastes quantités de données, d'apprendre des schémas, et de prendre des décisions autonomes, l'IA a le potentiel de débloquer de nouveaux niveaux d'efficacité, de productivité et d'innovation. Dans cet article de blog, nous explorerons le monde fascinant de l'Intelligence Artificielle, en examinant ses applications, son impact et ses perspectives futures.

## L'Impact de l'IA
L'IA a déjà fait des progrès significatifs dans divers domaines, de la santé et de la finance aux transports et au divertissement. En santé, les systèmes alimentés par l'IA peuvent analyser les images médicales, prédire les résultats des patients et aider au diagnostic et à la planification des traitements. En finance, les algorithmes d'IA peuvent analyser les tendances du marché, optimiser les portefeuilles d'investissement et détecter les activités frauduleuses. Dans les transports, l'IA pilote le développement de véhicules autonomes, révolutionnant notre manière de nous déplacer. Les possibilités sont infinies, et l'impact de l'IA ne fait que croître au fil des ans.

## L'Avenir de l'IA
Alors que l'IA continue d'évoluer, nous pouvons nous attendre à voir encore plus d'applications avancées et de capacités. Des expériences d'achat personnalisées et des assistants virtuels à la maintenance prédictive et aux villes intelligentes, l'IA jouera un rôle central dans la définition de l'avenir de notre société. Cependant, avec un grand pouvoir vient une grande responsabilité. Il est crucial de veiller à ce que les systèmes d'IA soient développés et déployés de manière éthique, en mettant l'accent sur la transparence, la responsabilité et l'équité. En embrassant le potentiel de l'IA tout en abordant ses défis, nous pouvons exploiter pleinement la puissance de cette technologie transformative pour le bénéfice de tous.

En conclusion, l'Intelligence Artificielle est un élément révolutionnaire qui a le potentiel de révolutionner notre manière de vivre et de travailler. En comprenant son impact, en embrassant ses possibilités et en abordant ses défis, nous pouvons débloquer un avenir alimenté par l'innovation de l'IA. Continuons à explorer les possibilités infinies de l'Intelligence Artificielle et œuvrons pour créer un monde où la technologie améliore nos vies de manière significative et positive.



> Entering new CrewAgentExecutor chain...
I now can give a great answer 

Final Answer:
# The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) has become an increasingly prevalent topic in today's society, with its influence reaching various aspects of our daily lives. From virtual assistants like Siri and Alexa to self-driving cars and predictive analytics, AI has transformed the way we interact with technology. However, with these advancements come concerns about the ethical implications and potential risks associated with AI.

One of the key benefits of AI is its ability to streamline processes and increase efficiency in various industries. For example, AI-powered algorithms can analyze large sets of data to make predictions and decisions faster and more accurately than humans. This has led to significant advancements in healthcare, finance, and other sectors, improving outcomes and reducing costs. Additionally, AI has the potential to create new jobs and

# The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) has become an increasingly prevalent topic in today's society, with its influence reaching various aspects of our daily lives. From virtual assistants like Siri and Alexa to self-driving cars and predictive analytics, AI has transformed the way we interact with technology. However, with these advancements come concerns about the ethical implications and potential risks associated with AI.

One of the key benefits of AI is its ability to streamline processes and increase efficiency in various industries. For example, AI-powered algorithms can analyze large sets of data to make predictions and decisions faster and more accurately than humans. This has led to significant advancements in healthcare, finance, and other sectors, improving outcomes and reducing costs. Additionally, AI has the potential to create new jobs and industries, driving economic growth and innovation.

On the other hand, there are concerns about the impact of AI on the job market and privacy rights. As AI technology continues to advance, there is a fear that automation will lead to job displacement and income inequality. Furthermore, the collection and use of personal data by AI systems raise questions about data privacy and security. It is crucial for policymakers and technology companies to address these issues and ensure that AI is developed and deployed responsibly. In conclusion, while AI offers immense potential for societal progress, it is essential to consider the ethical implications and risks associated with its widespread adoption.


=== FINAL (FR) ===


# L'impact de l'intelligence artificielle sur la société

L'intelligence artificielle (IA) est devenue un sujet de plus en plus prévalent dans la société d'aujourd'hui, son influence touchant divers aspects de notre vie quotidienne. Des assistants virtuels comme Siri et Alexa aux voitures autonomes et aux analyses prédictives, l'IA a transformé notre façon d'interagir avec la technologie. Cependant, ces avancées suscitent des inquiétudes concernant les implications éthiques et les risques potentiels associés à l'IA.

Un des principaux avantages de l'IA est sa capacité à rationaliser les processus et à accroître l'efficacité dans divers secteurs. Par exemple, les algorithmes alimentés par l'IA peuvent analyser de vastes ensembles de données pour faire des prédictions et des décisions plus rapidement et plus précisément que les humains. Cela a entraîné des progrès significatifs dans les secteurs de la santé, des finances et d'autres domaines, améliorant les résultats et réduisant les coûts. De plus, l'IA a le potentiel de créer de nouveaux emplois et industries, stimulant la croissance économique et l'innovation.

D'un autre côté, des préoccupations surgissent concernant l'impact de l'IA sur le marché du travail et les droits de la vie privée. Alors que la technologie de l'IA continue de progresser, il y a une crainte que l'automatisation entraîne des suppressions d'emplois et des inégalités de revenus. De plus, la collecte et l'utilisation des données personnelles par les systèmes d'IA soulèvent des questions sur la confidentialité et la sécurité des données. Il est essentiel que les décideurs politiques et les entreprises technologiques abordent ces problèmes et veillent à ce que l'IA soit développée et déployée de manière responsable. En conclusion, bien que l'IA offre un immense potentiel de progrès sociétal, il est essentiel de prendre en compte les implications éthiques et les risques associés à son adoption généralisée.